In [1]:
import json
import spacy
from bs4 import BeautifulSoup
nlp = spacy.load("output/model-best")

In [2]:
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

In [3]:
with open('./texts_for_testing/1880_90030.json', encoding="utf-8") as f:
    test_text = json.load(f)
opinion_of_text = test_text['html_lawbox']
cleaned_text_opinion = strip_html_tags(opinion_of_text)
print(cleaned_text_opinion[:500])


100 U.S. 158 (____)
Dow
v.
JOHNSON.
Supreme Court of United States.

The case was argued by The Attorney-General and Mr. E.B. Smith, Assistant Attorney-General, for the plaintiff in error, and by Mr. Thomas J. Durant for the defendant in error.
MR. JUSTICE FIELD delivered the opinion of the court.
The defendant in the court below, the plaintiff in error here, Neal Dow, was a brigadier-general in the army of the United States during the late civil war, and in 1862 and 1863 was *159 stationed in 


In [4]:
from pathlib import Path
folder_location = './texts_for_testing/'
paths = list(Path(folder_location).rglob('*.json'))
filenames = [k.stem for k in paths]
filenames

['1764_84587',
 '1783_84599',
 '1944_103915',
 '1880_90030',
 '1850_86508',
 '1764_84586',
 '1985_111301',
 '2022_opinion_2',
 '1865_87621',
 '1902_95542',
 '1764_2381788',
 '1920_99495',
 '1898_94785',
 '1783_84600',
 '1963_106601',
 '2022_opinion_1',
 '1804_84713']

In [5]:
names_and_scores = dict()

for file in paths:
    with open(file, encoding="utf-8") as f:
        data = json.load(f)
        stem = file.stem
        #stem = stem.split('_')[0]
    if data['plain_text'] != "":
        spacy_doc = nlp(data['plain_text'])
        names_and_scores[stem] = spacy_doc.cats
    elif data['html_with_citations'] != "":
        cleaned_text_opinion = strip_html_tags(data['html_with_citations'])
        spacy_doc = nlp(cleaned_text_opinion)
        names_and_scores[stem] = spacy_doc.cats
    elif data['html_lawbox'] != "":
        cleaned_text_opinion = strip_html_tags(data['html_lawbox'])
        spacy_doc = nlp(cleaned_text_opinion)
        names_and_scores[stem] = spacy_doc.cats

In [6]:
print(len(names_and_scores))

17


In [7]:
print(names_and_scores)

#13 out of 17 classified correctly ...:
13/17


{'1764_84587': {'1700': 1.0, '1800': 7.05525373366489e-13, '1900': 2.129194003996481e-14, '2000': 1.2912433510023388e-34}, '1783_84599': {'1700': 0.8163849115371704, '1800': 0.07637578248977661, '1900': 0.015045530162751675, '2000': 0.09219381958246231}, '1944_103915': {'1700': 0.0, '1800': 0.0, '1900': 1.0, '2000': 0.0}, '1880_90030': {'1700': 0.0, '1800': 1.0, '1900': 0.0, '2000': 0.0}, '1850_86508': {'1700': 0.9498764872550964, '1800': 0.05012354254722595, '1900': 5.0592957665505936e-20, '2000': 0.0}, '1764_84586': {'1700': 0.9999276399612427, '1800': 2.2554320821654983e-05, '1900': 4.986564090359025e-05, '2000': 8.872562240469506e-09}, '1985_111301': {'1700': 0.0, '1800': 0.0, '1900': 1.0, '2000': 0.0}, '2022_opinion_2': {'1700': 0.0, '1800': 0.0, '1900': 1.0, '2000': 0.0}, '1865_87621': {'1700': 4.1636241803644225e-05, '1800': 0.9999042749404907, '1900': 5.407072603702545e-05, '2000': 2.2901153610287632e-21}, '1902_95542': {'1700': 3.938906683731326e-26, '1800': 0.8571010231971741

0.7647058823529411

In [12]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.5f' % x)

df = pd.DataFrame.from_dict(names_and_scores).T
df.head()

,1700,1800,1900,2000
1764_84587,1.00000,0.00000,0.00000,0.00000
1783_84599,0.81638,0.07638,0.01505,0.09219
1944_103915,0.00000,0.00000,1.00000,0.00000
1880_90030,0.00000,1.00000,0.00000,0.00000
1850_86508,0.94988,0.05012,0.00000,0.00000


In [13]:
df['highest_score'] = df.idxmax(axis=1)

In [14]:
df

,1700,1800,1900,2000,highest_score
1764_84587,1.00000,0.00000,0.00000,0.00000,1700
1783_84599,0.81638,0.07638,0.01505,0.09219,1700
1944_103915,0.00000,0.00000,1.00000,0.00000,1900
1880_90030,0.00000,1.00000,0.00000,0.00000,1800
1850_86508,0.94988,0.05012,0.00000,0.00000,1700
1764_84586,0.99993,0.00002,0.00005,0.00000,1700
1985_111301,0.00000,0.00000,1.00000,0.00000,1900
2022_opinion_2,0.00000,0.00000,1.00000,0.00000,1900
1865_87621,0.00004,0.99990,0.00005,0.00000,1800
1902_95542,0.00000,0.85710,0.14290,0.00000,1800


In [11]:
'''
import dataframe_image as dfi
pd.set_option('display.float_format', lambda x: '%.5f' % x)

#dfi.export(df, 'test_table_png_file.png')
'''

"\nimport dataframe_image as dfi\npd.set_option('display.float_format', lambda x: '%.5f' % x)\n\n#dfi.export(df, 'test_table_png_file.png')\n"